In [ ]:
!git clone https://github.com/guanyuhoujeff/114_NKUST_Financial_Institution_Management.git

In [ ]:
cd /content/114_NKUST_Financial_Institution_Management/colab

In [1]:
import pandas as pd
import tqdm 
import os
import glob

# 現貨

In [2]:
full_spot = pd.read_csv('../data/full-spot.txt', encoding='cp950', delimiter='\t')
full_spot = full_spot.rename(columns={
    "證券代碼": "code",
    "簡稱" : "name",	
    "年月日" : "date",
    "收盤價(元)" : "close_price",
    # "報酬率％" : "spot_return",	
    "報酬率-Ln" : "spot_return"
})
full_spot['date'] = full_spot['date'].astype(int)
full_spot['spot_return'] = full_spot['spot_return'].astype(float)
full_spot['code'] = full_spot['code'].astype(str)

# 期貨

In [4]:
full_futures = pd.read_csv('../data/full-futures.txt', encoding='cp950', delimiter='\t')

nearby_full_futures_list = []
for date, group in full_futures.groupby("日期"):
    nearby_group = group[group["剩餘天數"] >  3]
    nearby_full_futures_list += nearby_group.sort_values(["剩餘天數"]).drop_duplicates("標的證券").to_dict("records")

nearby_full_futures_table = pd.DataFrame(nearby_full_futures_list)[['期貨名稱', '簡稱', '日期', '到期月', '報酬率', "標的證券"]]
nearby_full_futures_table = nearby_full_futures_table.rename(columns= {
    "簡稱": 'name',
    "日期": "date",
    "報酬率": "futures_return"
})
nearby_full_futures_table['code'] = nearby_full_futures_table["標的證券"].apply(lambda x: x.split()[0])
nearby_full_futures_table['date'] = nearby_full_futures_table['date'].astype(int)
nearby_full_futures_table['futures_return'] = nearby_full_futures_table['futures_return'].astype(float)

# SVI

In [5]:
svi_2542_table = pd.DataFrame()
for file in glob.glob("../data/SVI/興富發/*.csv"):
    df = pd.read_csv(file, header=1)
    svi_2542_table = pd.concat((svi_2542_table, df))
svi_2542_table = svi_2542_table.sort_values("天")
svi_2542_table.columns = ['date', "SVI"]
svi_2542_table["date"] = svi_2542_table["date"].apply(lambda x: int(x.replace("-", "")))
svi_2542_table["SVI"] = svi_2542_table["SVI"].astype(int)

In [6]:
svi_2515_table = pd.DataFrame()
for file in glob.glob("../data/SVI/中工/*.csv"):
    df = pd.read_csv(file, header=1)
    svi_2515_table = pd.concat((svi_2515_table, df))
svi_2515_table = svi_2515_table.sort_values("天")
svi_2515_table.columns = ['date', "SVI"]
svi_2515_table["date"] = svi_2515_table["date"].apply(lambda x: int(x.replace("-", "")))
svi_2515_table["SVI"] = svi_2515_table["SVI"].astype(int)

# 合併現貨-期貨-SVI

In [7]:
full_spot_futures = full_spot[['code', 'date', 'spot_return']].merge(nearby_full_futures_table[['code', 'date', 'futures_return']], on=['code', 'date'])
full_spot_futures = full_spot_futures[(full_spot_futures['date'] >= 20171003) & (full_spot_futures['date'] <= 20180614)]

In [8]:
#  SVI 虛擬變數 跟 交乘項
for code, svi_table in zip(["2515", "2542"], [svi_2515_table, svi_2542_table]):
    reserch_table = full_spot_futures[full_spot_futures["code"] == code].merge(svi_table, on='date').drop_duplicates("date")
    reserch_table["SVI_D1"] = (reserch_table["SVI"].quantile(0.25) > reserch_table["SVI"]).astype(int)
    reserch_table["SVI_D2"] = (reserch_table["SVI"].quantile(0.5) < reserch_table["SVI"]).astype(int)
    reserch_table["SVI_D3"] = (reserch_table["SVI"].quantile(0.75) < reserch_table["SVI"]).astype(int)
    # 交乘項
    reserch_table["d1_sr"] = reserch_table["SVI_D1"]  *  reserch_table["spot_return"] 
    reserch_table["d2_sr"] = reserch_table["SVI_D2"]  *  reserch_table["spot_return"] 
    reserch_table["d3_sr"] = reserch_table["SVI_D3"]  *  reserch_table["spot_return"] 

    reserch_table["d1_fr"] = reserch_table["SVI_D1"]  *  reserch_table["futures_return"] 
    reserch_table["d2_fr"] = reserch_table["SVI_D2"]  *  reserch_table["futures_return"] 
    reserch_table["d3_fr"] = reserch_table["SVI_D3"]  *  reserch_table["futures_return"] 

    # 匯出 excel
    reserch_table.to_excel(f"reserch_table_{code}.xlsx", index=False)
    print(f"存儲 excel 檔案： reserch_table_{code}.xlsx")

存儲 excel 檔案： reserch_table_2515.xlsx
存儲 excel 檔案： reserch_table_2542.xlsx
